In [1]:
!pip install chromadb
!pip install langchain
!pip install openai
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.1 MB/s eta 0:0

In [2]:
import chromadb

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
from openai import OpenAI

book_metadata =  {
        "title": "The Great Gatsby",
        "author" : "F. Scott Fitzgerald",
        "source_url": "https://www.gutenberg.org/cache/epub/64317/pg64317.txt",
        "filename": "pg64317.txt"
  }

In [9]:
client_openai = OpenAI(config.RAG_API_KEY)

#passing 2 dictionaries in messages because this is a convo between a system and a user
def get_completion(prompt):
    response = client_openai.chat.completions.create(
        model= "gpt-4",
        messages=[
            {"role":"system", "content": "You're a helpful assistant who retrieves information from external sources and presents them to the user."},
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message.content


#### Chunking the Document

In the next cell,
I am preparing the book and initializing a recursive text splitter and create chunks of our text with it.



In [10]:
import requests

In [11]:
r = requests.get(r'https://www.gutenberg.org/cache/epub/64317/pg64317.txt')
great_gatsby = r.text

In [12]:
great_gatsby = great_gatsby[1433:277912] #books w/o any footnotes
print(great_gatsby[:500]) #checking it

as Parke d’Invilliers


                                  I

In my younger and more vulnerable years my father gave me some advice
that I’ve been turning over in my mind ever since.

“Whenever you feel like criticizing anyone,” he told me, “just
remember that all the people in this world haven’t had the advantages
that you’ve had.”

He didn’t say any more, but we’ve always been unusually communicative
in a reserved way, and I understood that he meant a great deal more
than that. In


In [13]:
#examples of chunk size and chunk overlap
text_splitter = RecursiveCharacterTextSplitter(
    separators=[". ", "? ", "! "],
    chunk_size=2000,
    chunk_overlap=300,
)


chunks_gatsby = text_splitter.create_documents([great_gatsby])



In [14]:
print(f" 'The Great Gatsby' - First Chunk:\n{chunks_gatsby[0].page_content}\n")

 'The Great Gatsby' - First Chunk:
as Parke d’Invilliers


                                  I

In my younger and more vulnerable years my father gave me some advice
that I’ve been turning over in my mind ever since.

“Whenever you feel like criticizing anyone,” he told me, “just
remember that all the people in this world haven’t had the advantages
that you’ve had.”

He didn’t say any more, but we’ve always been unusually communicative
in a reserved way, and I understood that he meant a great deal more
than that. In consequence, I’m inclined to reserve all judgements, a
habit that has opened up many curious natures to me and also made me
the victim of not a few veteran bores. The abnormal mind is quick to
detect and attach itself to this quality when it appears in a normal
person, and so it came about that in college I was unjustly accused of
being a politician, because I was privy to the secret griefs of wild,
unknown men. Most of the confidences were unsought—frequently I have
feigne

Now let's create a new persistent Chroma collection.

Use Chroma's `.PersistentClient()` method to initialize a database that will persist throughout the program. Pass the method the desired route to the collection, `"./advanced"`.

Then use the Chroma client's `.get_or_create_collection()` method. Pass the method the `name` `"advanced"` and indicate i ll use cosine similarity with `metadata={"hnsw:space": "cosine"}`.


In [15]:
client_chroma = chromadb.PersistentClient(path="./advanced")
collection = client_chroma.get_or_create_collection(name = "advanced", metadata = {"hnsw:space": "cosine"})
#verifying my collection
print(f"ChromaDB collection {client_chroma.list_collections()}")

ChromaDB collection ['advanced']


#### Uploading the chunks

Now that my collection is initialized I can upload to it the chunks I made earlier.


In [16]:
#N = 150 #this was done because my kernel died persistently locally
#but in googlecollab everything is fine :)


for idx, chunk in enumerate(chunks_gatsby): #enumerating through a list of chunks
    doc_text = chunk.page_content #access the chunk's text
    book_metadata["chunk_idx"] = idx
    collection.add(
        documents=[doc_text],
        ids=[f"{book_metadata['title']}_{idx}"],
        metadatas=[book_metadata]
    )
#then adding a chunk index to its metadata and uploading the document, its id, and its metadata to Chroma collection.

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 28.4MiB/s]


#### Formatting the search results
Now let's define a helper function that takes a **user query** and returns a **well-formatted, pseudo-XML string of the search results**. This will make it easier to experiment

In [17]:
def populate_rag_query(query, n_results=1):
    search_results = collection.query(query_texts=[query], n_results=n_results)
    result_str = ""
    for idx, result in enumerate(search_results["documents"][0]):
        metadata = search_results["metadatas"][0][idx]
        formatted_result = f"""<SEARCH RESULT>
        <DOCUMENT>{result}</DOCUMENT>
        <METADATA>
        <TITLE>{metadata['title']}</TITLE>
        <AUTHOR>{metadata['author']}</AUTHOR>
        <CHUNK_IDX>{metadata['chunk_idx']}</CHUNK_IDX>
        <URL>{metadata['source_url']}</URL>
        </METADATA>
        </SEARCH RESULT>"""
        result_str += formatted_result
    return result_str

Finally, create the RAG prompt to send to the LLM.

In the `<INSTRUCTIONS>` section, we need to consider how we might guide the model to:
 - Use the search results effectively
 - Handle cases where information isn't available
 - Provide credibility to its answers by citing sources

In the  `<EXAMPLE CITATION>`, we  can show the model how its cited sources should look

To wrap it up, we then pass the correct variables in the `<USER QUERY>` and `<SEARCH RESULTS>` sections to finish the function.



In [18]:
def make_rag_prompt(query, results):
    return f"""<INSTRUCTIONS>
   <EXAMPLE CITATION>
   Answer to the user query in your own words, drawn from the search results.
   - "Direct quote from source material backing up the claim" - [Source: Title, Author, Chunk: chunk index, Link: url]
   </EXAMPLE CITATION>
   </INSTRUCTIONS>

    <USER QUERY>
    {query}
    </USER QUERY>

    <SEARCH RESULTS>
    {results}
    </SEARCH RESULTS>

    Your answer:"""

In [19]:
#get method is used to retrieve info from a collection

collection.get('The Great Gatsby_10')

{'ids': ['The Great Gatsby_10'],
 'embeddings': None,
 'documents': ['. Her grey sun-strained eyes looked\r\nback at me with polite reciprocal curiosity out of a wan, charming,\r\ndiscontented face. It occurred to me now that I had seen her, or a\r\npicture of her, somewhere before.\r\n\r\n“You live in West Egg,” she remarked contemptuously. “I know somebody\r\nthere.”\r\n\r\n“I don’t know a single—”\r\n\r\n“You must know Gatsby.”\r\n\r\n“Gatsby?” demanded Daisy. “What Gatsby?”\r\n\r\nBefore I could reply that he was my neighbour dinner was announced;\r\nwedging his tense arm imperatively under mine, Tom Buchanan compelled\r\nme from the room as though he were moving a checker to another square.\r\n\r\nSlenderly, languidly, their hands set lightly on their hips, the two\r\nyoung women preceded us out on to a rosy-coloured porch, open toward\r\nthe sunset, where four candles flickered on the table in the\r\ndiminished wind.\r\n\r\n“Why candles?” objected Daisy, frowning. She snapped the

In [20]:
collection.get(where={"chunk_idx": { "$eq":151}})
#with the unexistent id and it will return everything empty

{'ids': ['The Great Gatsby_151'],
 'embeddings': None,
 'documents': ['. “How about saying you’ll come?”\r\n\r\n“Well, the fact is—the truth of the matter is that I’m staying with\r\nsome people up here in Greenwich, and they rather expect me to be with\r\nthem tomorrow. In fact, there’s a sort of picnic or something. Of\r\ncourse I’ll do my best to get away.”\r\n\r\nI ejaculated an unrestrained “Huh!” and he must have heard me, for he\r\nwent on nervously:\r\n\r\n“What I called up about was a pair of shoes I left there. I wonder if\r\nit’d be too much trouble to have the butler send them on. You see,\r\nthey’re tennis shoes, and I’m sort of helpless without them. My\r\naddress is care of B. F.—”\r\n\r\nI didn’t hear the rest of the name, because I hung up the receiver.\r\n\r\nAfter that I felt a certain shame for Gatsby—one gentleman to whom I\r\ntelephoned implied that he had got what he deserved. However, that was\r\nmy fault, for he was one of those who used to sneer most bitterly 

In [21]:
collection.get(where={"chunk_idx": { "$eq":10}})
#another way of getting info from the collection

{'ids': ['The Great Gatsby_10'],
 'embeddings': None,
 'documents': ['. Her grey sun-strained eyes looked\r\nback at me with polite reciprocal curiosity out of a wan, charming,\r\ndiscontented face. It occurred to me now that I had seen her, or a\r\npicture of her, somewhere before.\r\n\r\n“You live in West Egg,” she remarked contemptuously. “I know somebody\r\nthere.”\r\n\r\n“I don’t know a single—”\r\n\r\n“You must know Gatsby.”\r\n\r\n“Gatsby?” demanded Daisy. “What Gatsby?”\r\n\r\nBefore I could reply that he was my neighbour dinner was announced;\r\nwedging his tense arm imperatively under mine, Tom Buchanan compelled\r\nme from the room as though he were moving a checker to another square.\r\n\r\nSlenderly, languidly, their hands set lightly on their hips, the two\r\nyoung women preceded us out on to a rosy-coloured porch, open toward\r\nthe sunset, where four candles flickered on the table in the\r\ndiminished wind.\r\n\r\n“Why candles?” objected Daisy, frowning. She snapped the

# Decoupling
what is decoupling? it is the process when we seperate the query embedding from the returned search results

Why?
we make sure that these two parts can be optimized separately. This is good for perfomance as our **retrieval** model will focus entirely on finding the most relevant search results, and the **generation** model can focus on generating "good" responses based on the retrieved data

In [22]:
#this function takes a chunk index of the retrieved search results
#then it return the previous and next chunks
def get_next_and_previous_chunks(chunk_idx):
  previous_chunk = collection.get(where = {"chunk_idx": {"$eq": chunk_idx - 1}})
  next_chunk =  collection.get(where = {"chunk_idx": {"$eq": chunk_idx + 1}})
 #passing chunk_idx to the where argument

  return previous_chunk, next_chunk

In [25]:
#this function accepts the original chink and returns a string of search reuslts for the previous, current, and next chunks
def expanded_search_results(original_chunk):
    original_chunk_idx = original_chunk["metadatas"][0]["chunk_idx"]
    previous_chunk, next_chunk = get_next_and_previous_chunks(original_chunk_idx)
    result_str = ""
    for chunk in [previous_chunk, original_chunk, next_chunk]:
        if len(chunk["metadatas"])>0:
            metadata = chunk["metadatas"][0]
            formatted_result = f"""<SEARCH RESULT>
            <DOCUMENT>{chunk["documents"][0]}</DOCUMENT>
            <METADATA>
            <TITLE>{metadata["title"]}</TITLE>
            <AUTHOR>{metadata["author"]}</AUTHOR>
            <CHUNK_IDX>{metadata["chunk_idx"]}</CHUNK_IDX>
            <URL>{metadata["source_url"]}</URL>
            </METADATA>
            </SEARCH RESULT>"""
            result_str += formatted_result
    return result_str
original_demo_chunk = collection.get(where={"chunk_idx": {"$eq": 10}})
print("Let's see what we have ")
expanded_results = expanded_search_results(original_demo_chunk)
print(expanded_results)

Let's see what we have 
<SEARCH RESULT>
            <DOCUMENT>. Tomorrow!” Then she added
irrelevantly: “You ought to see the baby.”

“I’d like to.”

“She’s asleep. She’s three years old. Haven’t you ever seen her?”

“Never.”

“Well, you ought to see her. She’s—”

Tom Buchanan, who had been hovering restlessly about the room, stopped
and rested his hand on my shoulder.

“What you doing, Nick?”

“I’m a bond man.”

“Who with?”

I told him.

“Never heard of them,” he remarked decisively.

This annoyed me.

“You will,” I answered shortly. “You will if you stay in the East.”

“Oh, I’ll stay in the East, don’t you worry,” he said, glancing at
Daisy and then back at me, as if he were alert for something
more. “I’d be a God damned fool to live anywhere else.”

At this point Miss Baker said: “Absolutely!” with such suddenness that
I started—it was the first word she had uttered since I came into the
room. Evidently it surprised her as much as it did me, for she yawned
and with a series of rapid

In [26]:
def make_decoupled_rag_prompt(query, n_results=1):

    search_results = collection.query(query_texts=[query], n_results=n_results)
    total_result_str = ""
    for doc, metadata in zip(search_results['documents'][0], search_results['metadatas'][0]):
        chunk = {
            'documents': [doc],
            'metadatas': [metadata]
        }

        expanded_result = expanded_search_results(chunk)
        total_result_str += expanded_result
        rag_prompt = make_rag_prompt(query, total_result_str)
    return rag_prompt

# given a search result, we can get everything back in a formatted string
prompt = make_decoupled_rag_prompt("How are West Egg and East Egg different?")
rag_completion = get_completion(prompt)
print(rag_completion)
#

prompt_1 = make_decoupled_rag_prompt("How does Nick Carraway first meet Jay Gatsby?")
rag_completion_1 = get_completion(prompt_1)
print(rag_completion_1)
#perfect! if you have read the book, these answers are correct hehe

West Egg and East Egg, depicted in F. Scott Fitzgerald's "The Great Gatsby," are two imaginary communities located on Long Island. They differ primarily in terms of social class and societal perception. West Egg is less fashionable and is inhabited by the nouveau riche, or those with newly acquired wealth, marked by their extravagance and lack of established social connections. This is where the narrator lives, as well as Gatsby, whose incredible mansion exemplifies the excesses of the inhabitants. East Egg, on the other hand, is depicted as more fashionable and is populated by older, established wealth. The inhabitants, like the Buchanans, uphold the societal conventions of the upper class. East Egg is characterized by its white palaces glittering along the water, and is the embodiment of elegance and class. - [Source: The Great Gatsby, F. Scott Fitzgerald, Chunk: 4, Link: https://www.gutenberg.org/cache/epub/64317/pg64317.txt]
Nick Carraway first meets Jay Gatsby during one of Gatsby

# Contextual Query Retrieval
this improves the retrieval part of the rag process by understanding the context of the convo

cqr also takes into the account the whole convo history when fetching relevant info rather than treating each query in isolation

why?
improving the relevance and coherence of the retrieved data
more accurate responses

In [27]:
#first create imaginary chat entries
#contextual_query relies on the chat history

chat_memory = [
    {"role": "user", "content": "How are West Egg and East Egg different?"},
    {"role": "assistant", "content": "West Egg and East Egg, depicted in F. Scott Fitzgerald's 'The Great Gatsby,' are two imaginary communities located on Long Island. They differ primarily in terms of social class and societal perception. West Egg is less fashionable and is inhabited by the nouveau riche, or those with newly acquired wealth, marked by their extravagance and lack of established social connections. This is where the narrator lives, as well as Gatsby, whose incredible mansion exemplifies the excesses of the inhabitants. East Egg, on the other hand, is depicted as more fashionable and is populated by older, established wealth. The inhabitants, like the Buchanans, uphold the societal conventions of the upper class. East Egg is characterized by its white palaces glittering along the water, and is the embodiment of elegance and class."},
    {"role": "user", "content": "How does Gatsby's house look like?"},
    {"role": "assistant", "content": "The [house] on my right was a colossal affair by any standard–it was a factual imitation of some Hôtel de Ville in Normandy, with a tower on one side, spanking new under a thin beard of raw ivy, and a marble swimming pool and more than forty acres of lawn and garden"},
    {"role": "user", "content": "Why does Gatsby stop throwing parties?"},
    {"role":"assistant", "content": "Early in the novel, when Jordan explains Gatsby’s relationship with Daisy to Nick, she says that Gatsby “half-expected [Daisy] to wander into one of his parties, some night.” After Daisy and Tom finally attend one of Gatsby’s parties, Gatsby abruptly stops throwing them, for there no longer is a reason to hold such events. Now that both Daisy and Tom have seen proof of Gatsby’s wealth, he doesn’t feel the need to show it off anymore."}
    ]

contextual_query = "What was the relationship between Gatsby and Daisy?"

In [28]:
#now i need to concatenate the chat history to be easily injected into a preliminary llm prompt

chat_history = ""

for message in chat_memory:
  role = message["role"]
  content = message["content"]
  chat_history += f"{role}: {content}\n\n"

print("This is my chat history: ")
print(chat_history)

This is my chat history: 
user: How are West Egg and East Egg different?

assistant: West Egg and East Egg, depicted in F. Scott Fitzgerald's 'The Great Gatsby,' are two imaginary communities located on Long Island. They differ primarily in terms of social class and societal perception. West Egg is less fashionable and is inhabited by the nouveau riche, or those with newly acquired wealth, marked by their extravagance and lack of established social connections. This is where the narrator lives, as well as Gatsby, whose incredible mansion exemplifies the excesses of the inhabitants. East Egg, on the other hand, is depicted as more fashionable and is populated by older, established wealth. The inhabitants, like the Buchanans, uphold the societal conventions of the upper class. East Egg is characterized by its white palaces glittering along the water, and is the embodiment of elegance and class.

user: How does Gatsby's house look like?

assistant: The [house] on my right was a colossal a

In [29]:
#prompting the model to rewrite the query given its chat context
def rewrite_query(query, chat_history):
  prompt = f"""<INSTRUCTIONS>
Given the following chat history and the user's latest query, rewrite the query to include relevant context.
</INSTRUCTIONS>

<CHAT_HISTORY>
{chat_history}
</CHAT_HISTORY>

<LATEST_QUERY>
{query}
</LATEST_QUERY>

Your rewritten query:"""

  return get_completion(prompt)

cqr_query = rewrite_query(contextual_query, chat_history)

print("original query: ")
print(contextual_query)
print("rewritten query: ")
print(cqr_query)


original query: 
What was the relationship between Gatsby and Daisy?
rewritten query: 
Can you describe the relationship between Gatsby and Daisy in 'The Great Gatsby'?


In [30]:
# refine the query using the chat history
# 2. refined query is used to populate the search results
# 3. convert to a rag prompt
#4. send to llm api

def perform_cqr_rag(query, chat_history, n_results=2):
    refined_query = rewrite_query(query, chat_history)
    result_str = populate_rag_query(refined_query, n_results)
    rag_prompt = make_rag_prompt(refined_query, result_str)
    rag_completion = get_completion(rag_prompt)
    return refined_query, rag_completion

refined_query, rag_completion = perform_cqr_rag(contextual_query, chat_history)
print(rag_completion)


The relationship between Gatsby and Daisy in F. Scott Fitzgerald's 'The Great Gatsby' is a complicated one, marked by romance, secrecy, and tension. Gatsby harbors a deep love for Daisy and is invested in keeping their interactions hidden from others, as evidenced by him not wanting Daisy to know about him asking others to invite her to tea. Further evidence of their romantic connection is seen in the intense and profound moment when Daisy tells Gatsby that she loves him, revealing the affection that exists between them. However, this also results in tension as Daisy's husband Tom is present, and visibly startled by this admission, implying a love triangle that complicates the relationship between Daisy and Gatsby. 
- "“Does she want to see Gatsby?” / “She’s not to know about it. Gatsby doesn’t want her to know. You’re just supposed to invite her to tea.”" - [Source: The Great Gatsby, F. Scott Fitzgerald, Chunk: 72, Link: https://www.gutenberg.org/cache/epub/64317/pg64317.txt]
- "She h

# Hypothetical document embeddings
this is an approach to the retrieval process in rag systems

in the standard flow, we embed the user's query directly for similarity search

with hyde, we first ask the llm to generate a hypothetical answer based on the query. then, this answer is embedded and used for similarity search

why?
the model's answer often contains terms that are likely to appear in relevant doc chunks, potnetially leading to more accurate retrievals than the orig query alone

In [31]:
#first, create a prompt that will generate a hypothetical doc
def make_hyde_prompt(query):
  return f"""<INSTRUCTIONS>
You are given the user's query. First, try to answer the query on your own. If the answer is not in your database, imitate the answer like you know it. Think of the language that might appear in the actual answers, and use it. Finally, the size of your answer should be a paragraph long.
</INSTRUCTIONS>

<QUERY>{query}</QUERY>
"""


In [32]:
query = "How does Gatsby's house look like?"
hyde_query_prompt = make_hyde_prompt(query)
hyde_query = get_completion(hyde_query_prompt) #the model's attempt to answer the question, even without having access to the actual information
print(hyde_query)

#search the doc chunks stored in the collection for the most relevant info
results = collection.query(query_texts=[hyde_query], n_results=3)
print(results)


In F. Scott Fitzgerald's novel "The Great Gatsby," Jay Gatsby's house is described as a colossal affair that is a classic example of the extravagant lifestyle in the 1920s. The mansion is situated in the village of West Egg on Long Island and it is described as a place of old wealth and sophistication. The house is depicted as having a Gothic style, furnished lavishly with period pieces, and even has a tower on one side. The estate boasts a vast lawn with gardens and is set right on the water, complete with its own private beach, a dock for his hydroplane, and a sparkling marina. Inside, the house is meticulously laid out with elegant rooms, including a grand ballroom where Gatsby holds his lavish parties. Despite the excellence and wealth illustrated by the house, it is an emblem of loneliness and a symbol of Gatsby's unfulfilled dream to win Daisy's love back.
{'ids': [['The Great Gatsby_72', 'The Great Gatsby_4', 'The Great Gatsby_10']], 'embeddings': None, 'documents': [['. Suddenl

#what happened:
This hypothetical answer is used as the basis for the search. The aim is to retrieve more relevant document chunks by using a richer query representation.


In [33]:
def answer_query_with_hyde(user_query):
  hyde_prompt = make_hyde_prompt(user_query)
  hyde_query = get_completion(hyde_prompt)
  result_str = populate_rag_query(hyde_query, n_results=3)
  rag_prompt = make_rag_prompt(hyde_query, result_str)
  rag_completion = get_completion(rag_prompt)
  return rag_completion

user_query = "How does Gatsby's house look like?"
answer = answer_query_with_hyde(user_query)
print(answer)

In "The Great Gatsby" by F. Scott Fitzgerald, Gatsby's mansion is located in West Egg on Long Island, and its grandeur encapsulates his immense wealth and his desire to flaunt it. The house is an "imitation of some Hôtel de Ville in Normandy" with a tower on one side, a thin layer of raw ivy implying its newness, a marble swimming pool, and more than forty acres of lawn and garden. This demonstrates Gatsby's opulence and his ambition to impress the high society. Inside, the mansion comprises a library filled with actual books indicating Gatsby's strive to portray himself as traditionally educated and refined. His house, therefore, is a symbol of his aspirations, illusions, and ultimately, his catastrophes. The visitors to Gatsby's house during a particular summer include people from both West and East Egg, signifying acceptance and recognition of his status from high society, despite their knowing little about him. - [Source: The Great Gatsby, F. Scott Fitzgerald, Chunk: 4 & 54, Link: 

#Fusion search

method to handle complex queries

it breaks down a complex query into miltiple subqueries
each of them is processed individually, and then when it comes to results, they are combined


In [35]:
import json

In [36]:
response = get_completion("""<INSTRUCTIONS>
Generate valid JSON with the key "data" and a list of 1,2,3.
Only generate the JSON. Do not output any additional characters.
Do not output markdown backticks. Just raw JSON.
</INSTRUCTIONS>

YOUR JSON:""")
our_loaded_json = json.loads(response)
for number in our_loaded_json['data']:
    print(f"Number: {number}")

Number: 1
Number: 2
Number: 3


In [37]:
def generate_subquestions(query):
    prompt = f"""<INSTRUCTIONS>
Given the following user query, generate a list of 2-4 subquestions that would help in answering the original query.
Return the result as JSON with the key "data" and the list as its value.
Only output valid JSON and no other characters.
Do not output markdown backticks. Just output raw JSON only.
</INSTRUCTIONS>

<QUERY>{query}</QUERY>
"""
    response = get_completion(prompt)
#     if the model generates invalid json, uncomment the print line and inspect what went wrong
#     print(response)
    subquery_dict = json.loads(response)
    return subquery_dict['data']

generate_subquestions("What should I eat for dinner tonight?")

['What dietary restrictions or preferences do you have?',
 'Are you looking for a recipe to cook at home or a recommendation for restaurant dining?',
 'Are there any specific types of cuisine that you prefer?',
 'Are you currently following any specific diet?']

In [41]:
def get_and_concat_subquestions(query):
    subquestions = generate_subquestions(query)
    subquestion_context = []
    for subquestion in subquestions:
        result_str = populate_rag_query(subquestion)
        rag_prompt = make_rag_prompt(subquestion, result_str)
        answer = get_completion(rag_prompt)
        subquestion_context.append(f"Q: {subquestion}\nA: {answer}")
    return "\n\n".join(subquestion_context)

In [42]:
def fusion_search(query):
    print(f"Original query: {query}")
    subquestion_context = get_and_concat_subquestions(query)
    print("Subquestion context:", subquestion_context)
    final_prompt = f"""<INSTRUCTIONS>
Using the following information from subquestions, answer the original query.
</INSTRUCTIONS>

<SUBQUESTION_INFO>
{subquestion_context}
</SUBQUESTION_INFO>

<ORIGINAL_QUERY>{query}</ORIGINAL_QUERY>

Final Answer:"""

    final_answer = get_completion(final_prompt)
    return final_answer

user_query = "Why did Daisy marry Tom even though she has feelings to Gatsby?"
result = fusion_search(user_query)

print("\nFinal Result:")
print(result)

Original query: Why did Daisy marry Tom even though she has feelings to Gatsby?
Subquestion context: Q: What is the relationship between Daisy, Tom and Gatsby in the storyline?
A: In the storyline of "The Great Gatsby" by F. Scott Fitzgerald, Daisy and Tom Buchanan are married, but Daisy and Gatsby share a romantic connection. This creates a complex love triangle. In a particular scene, Daisy openly tells Gatsby that she loves him, while her husband Tom is present. This revelation astounds Tom. The tension between the three characters grows as Tom observes the undeniable chemistry between his wife and Gatsby. [Source: The Great Gatsby, F. Scott Fitzgerald, Chunk: 105, Link: https://www.gutenberg.org/cache/epub/64317/pg64317.txt]

Q: What are the key events that led to Daisy's decision to marry Tom?
A: Daisy's decision to marry Tom Buchanan followed a series of events. After presumably being prevented from going to New York to say goodbye to a soldier going overseas, Daisy stopped mingl